In [ ]:
#This script uses over 300 GB of RAM

#Tbis notebook loads data and then uses leave-one-region-out cross-validation, to train 28 models, 
#doing three repeats of each
#Each model takes about 10 minutes to train


In [ ]:
#before starting: update base folder name
base_folder = '../../'

In [ ]:
import os
#select a GPU
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

import copy
import sys
import time
import numpy as np
import pandas as pd

sys.path.append('../SemanticSegmenterCNN') # for P4_DataHandlers

#imports
from p4tools import io # from https://github.com/michaelaye/p4tools: 
from P4_DataHandlers import Load_P4_Data_PreSaved,CreateFullImageMasks,CreateListOfThreeScales,Load_all_HiRISE_images_crop_and_convert_to_8bits

#keras and tensorflow imports needed for specifying and training a model
import tensorflow
print("tensorflow version = ",tensorflow.__version__)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.layers import Input,GlobalAveragePooling2D,Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.utils import Sequence

In [ ]:
#classifier parameters
DoScaleAug = True #set this to false unless you have 400 GB of RAM!
num_classes = 2
init_lr = 0.001
num_epochs = 10
FirstEpochStep=9
SecondEpochStep=20
batch_size = 16
My_wd = 5e-4
NumRepeats=3

In [ ]:
#get meta data information
Tiles_df = io.get_tile_coordinates()
print("Number of HiRise full Images = ", Tiles_df['obsid'].unique().shape[0])

#regions
region_names_df = io.get_region_names()
#add some missing region names
region_names_df = region_names_df.set_index('obsid')
region_names_df.at['ESP_012620_0975','roi_name'] = 'Buffalo'
region_names_df.at['ESP_012277_0975','roi_name'] = 'Buffalo'
region_names_df.at['ESP_012348_0975','roi_name'] = 'Taichung'

#metadata
metadata_df = io.get_meta_data()
for index, row in metadata_df.iterrows():
    roi_name = region_names_df.at[row['OBSERVATION_ID'],'roi_name']
    metadata_df.at[index,'roi_name']=roi_name
    
#ensure we use only 
ImageResults_df = io.get_meta_data()
ImageResults_df = ImageResults_df.set_index('OBSERVATION_ID')
ImageResults_df = pd.concat([ImageResults_df, region_names_df], axis=1, sort=False)
ImageResults_df=ImageResults_df.dropna()

UniqueP4Regions = ImageResults_df['roi_name'].unique()
UniqueP4Regions = metadata_df['roi_name'].unique()
print("Number of P4 regions = ",len(UniqueP4Regions))


In [ ]:
#prepare data for training models (create 3 scales for each, to speed up training. Uses lots of RAM)
t0 = time.time()
ImageAndMasksList=[]
OBSID_List=[]
DataFolder = base_folder+'/Data/Images/HiRISE_8bit_and_P4_mask/'

Preload=False
if Preload==False:
    #load all the images and convert from 10 bits maximum bit depth per channel to 8. Then crop to the labelled region
    #Takes about 70 GB of RAM
    t0 = time.time()
    ImageAndMasksList,OBSID_List = Load_all_HiRISE_images_crop_and_convert_to_8bits(base_folder+'Data/Images/HiRISE/') #result has 3 channels (RGB)
    t1 = time.time()
    print('Number of images loaded = ',len(ImageAndMasksList))
    print("time to load and convert all images = ",t1-t0) #takes about 2 hours
    #step 3: create "ground truth" segmentation masks.
    t0 = time.time()
    ImageAndMasksList =  CreateFullImageMasks(ImageAndMasksList) # Result has 4 channels ( RGB and the 4th dimension in the channels axis is the masks)
    t1 = time.time()
    print("time to create all masks = ",t1-t0)#takes 2-3 hours
    if not os.path.exists(DataFolder):
        os.makedirs(DataFolder)  
    for i in range(len(ImageAndMasksList)):
        obs_id = OBSID_List[i]
        np.save(DataFolder+obs_id+'.npy', ImageAndMasksList[i])
else:
    #assumes the first if branch above has already been run
    for index, row in metadata_df.iterrows():
        OBSID_List.append( row['OBSERVATION_ID'])
        ImageAndMasksList.append(np.load(DataFolder+row['OBSERVATION_ID']+'.npy'))
    t1 = time.time()
    print("time to load all presaved images = ",t1-t0) 
    print('Number of images loaded = ',len(ImageAndMasksList))
    
assert len(ImageAndMasksList)==Tiles_df['obsid'].unique().shape[0]

if DoScaleAug == True:
    t0 = time.time()
    ImageAndMasksLists = CreateListOfThreeScales(ImageAndMasksList) #takes 1.5-2 hours
    t1 = time.time()
    print('time to createlist of three scales for scale augmentation = ',t1-t0) 
else:
    ImageAndMasksLists=[ImageAndMasksList]
 

In [ ]:
def metric_balanced_accuracy_score(y_true, y_pred):
    from sklearn.metrics import balanced_accuracy_score
    import tensorflow as tf
    from tensorflow.keras import backend as K
    return tf.numpy_function(balanced_accuracy_score, (K.argmax(y_true,axis=-1), K.argmax(y_pred,axis=-1)), tf.double)


In [ ]:
class P4_TileClassifierDataGenerator(Sequence):
    def __init__(self, ImageLists,batch_size,patch_height,patch_width):
        self.ImageLists = ImageLists #this is a list of lists of training images at different scales
        self.batch_size = batch_size
        self.patch_height = patch_height
        self.patch_width = patch_width
        self.ordering = np.arange(len(self.ImageLists[0]))
        np.random.shuffle(self.ordering) 
    def on_epoch_end(self):
        np.random.shuffle(self.ordering)
    def __len__(self):
        return 35*int(np.floor(len(self.ImageLists[0])/self.batch_size)) #35 is in the order of the number of tiles per image
    def __getitem__(self, index):
        #the data coming in has 4 channels (RGB plus mask). Do augmentation first and only separate for return
        imgs=[]
        labels=[]
        for i in range(self.batch_size):
            #randomly choose from the scales available in the list of image lists
            WhichScale=np.random.randint(len(self.ImageLists))            
            img=self.ImageLists[WhichScale][self.ordering[(index*self.batch_size+i)%35]]
            #get a random location within the image
            start_height=np.random.randint(img.shape[0]-self.patch_height)
            start_width=np.random.randint(img.shape[1]-self.patch_width)
            #get the patch
            ThisPatch=copy.deepcopy(img[start_height:start_height+self.patch_height,start_width:start_width+self.patch_width,:])
            #add random flips and 90 degree rotations
            if np.random.rand(1)>0.5:
                ThisPatch=np.flipud(ThisPatch)
            if np.random.rand(1)>0.5:
                ThisPatch=np.fliplr(ThisPatch)
            if np.random.rand(1)>0.5:
                ThisPatch=np.rot90(ThisPatch)
            imgs.append(ThisPatch)
            labels.append(np.amax(ThisPatch[:,:,-1])>0)
        #convert list to numpy
        imgs=np.stack(imgs,axis=0)
        labels=np.array(labels)
        #return the image channels and categorical mask representations separately
        return imgs[:,:,:,0:3],tensorflow.keras.utils.to_categorical(labels,2)

In [ ]:
#learning rate schedule
def lr_schedule(epoch):
    lr = init_lr
    if epoch >= SecondEpochStep:
        lr *= 1e-2
    elif epoch >= FirstEpochStep:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr
    
#loop over each region, and leave it out of training for a model. Use the left out region for val
UniqueP4Regions = metadata_df['roi_name'].unique()

ModelsPath='../../Data/Models/TileClassifier/'
if not os.path.isdir(ModelsPath):
    os.mkdir(ModelsPath)

for Repeat in range(NumRepeats):
    ModelCount = 0
    for ToLeaveOut in UniqueP4Regions:
        #select data for training
        LeaveOneRegionOutList = [n for n in UniqueP4Regions if n != ToLeaveOut]
        print('Training model ',str(ModelCount+1),' of ',str(len(UniqueP4Regions)),'; leaving out region: ',ToLeaveOut)
        TrainImages_df = metadata_df[metadata_df['roi_name'] != ToLeaveOut]
        if DoScaleAug==True:
            TrainImagesList1 = [ImageAndMasksLists[0][i] for i in  TrainImages_df.index.values]
            TrainImagesList2 = [ImageAndMasksLists[1][i] for i in  TrainImages_df.index.values]
            TrainImagesList3 = [ImageAndMasksLists[2][i] for i in  TrainImages_df.index.values]
            TrainImagesList = [TrainImagesList1,TrainImagesList2,TrainImagesList3]
        else:
            TrainImagesList = [[ImageAndMasksLists[0][i] for i in  TrainImages_df.index.values]]
        train_data_generator = P4_TileClassifierDataGenerator(TrainImagesList,batch_size,384,384)
        #define the model - we use transfer learning, based on a ResNet50 pretrained on imagenet
        base_model = ResNet50(include_top=False, weights='imagenet')
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        predictions = Dense(num_classes, activation='softmax',
                        kernel_initializer='he_normal',
                        kernel_regularizer=l2(My_wd),use_bias=False)(x)
        model = Model(inputs=base_model.input, outputs=predictions)
        model.compile(loss='categorical_crossentropy',
                  optimizer =SGD(lr=init_lr, decay=0, momentum=0.9, nesterov=False),
                  metrics=['accuracy',metric_balanced_accuracy_score])
        #fit the model
        lr_scheduler = LearningRateScheduler(lr_schedule)
        history = model.fit_generator(train_data_generator,
                                      epochs=num_epochs,
                                      verbose=1, 
                                      workers=8,
                                      max_queue_size=50,
                                      callbacks=[lr_scheduler]
                                     )
        model.save(ModelsPath+'HiRISE_tile_classifier_leave_out_' + ToLeaveOut+ '_final_repeat'+str(Repeat)+'.h5') 
        ModelCount=ModelCount+1

In [ ]:
#validation
ResultsPath='../../Data/SummaryResults/'
if not os.path.isdir(ResultsPath):
    os.mkdir(ResultsPath)

for Repeat in range(NumRepeats):
    ModelCount = 0
    Results_df = pd.DataFrame()
    for ToLeaveOut in UniqueP4Regions:
        LeaveOneRegionOutList = [n for n in UniqueP4Regions if n != ToLeaveOut]
        print('model ',str(ModelCount+1),' of ',str(len(UniqueP4Regions)),'; validation region: ',ToLeaveOut)
        ValImages_df = metadata_df[metadata_df['roi_name'] == ToLeaveOut]
        ValImagesList=[]
        for i in range(ValImages_df.shape[0]):
            if ValImages_df.iloc[i]['map_scale']==0.25:
                ScaleInd=2
            elif ValImages_df.iloc[i]['map_scale']==0.5:
                ScaleInd=1
            else:
                ScaleInd=0
            ValImagesList.append(ImageAndMasksLists[ScaleInd][ValImages_df.index[i]])
        model.load_weights(ModelsPath+'HiRISE_tile_classifier_leave_out_' + ToLeaveOut+ '_final_repeat'+str(Repeat)+'.h5') 
        CC=0
        for Im in ValImages_df['OBSERVATION_ID']:
            ThisImageTiles = Tiles_df[Tiles_df['obsid']==Im]
            ThisImageTiles=ThisImageTiles.reset_index(drop=True)
            ThisImageTiles['y_hirise']=ThisImageTiles['y_hirise'].astype('uint32')
            ThisImageTiles['x_hirise']=ThisImageTiles['x_hirise'].astype('uint32')
            FullIm=ValImagesList[CC][:,:,0:3]
            FullMask=ValImagesList[CC][:,:,3]
            Tiles=[]
            for i in range(ThisImageTiles.shape[0]):
                Results_df.at[Im+'_'+str(i),'tile_id']= ThisImageTiles.at[i,'tile_id']
                Results_df.at[Im+'_'+str(i),'Region']=ToLeaveOut
                Start_col =int(ThisImageTiles.at[i,'x_hirise']-420)
                Start_row =int(ThisImageTiles.at[i,'y_hirise']-324)
                TileImage=FullIm[Start_row:min(Start_row+648,FullIm.shape[0]),Start_col:min(Start_col+840,FullIm.shape[1]),:]
                TileMask = FullMask[Start_row:min(Start_row+648,FullIm.shape[0]),Start_col:min(Start_col+840,FullIm.shape[1])]
                Results_df.at[Im+'_'+str(i),'Region']=ToLeaveOut
                Results_df.at[Im+'_'+str(i),'GroundTruth']=np.amax(TileMask)
                if TileImage.shape[0] != 648 or TileImage.shape[1] != 840:
                    aaa=255*np.ones((648,840,3),'uint8')
                    aaa[0:TileImage.shape[0],0:TileImage.shape[1],:]=TileImage
                    TileImage=aaa
                Tiles.append(TileImage)
            Tiles=np.stack(Tiles,axis=0)
            PredClass = model.predict(Tiles)
            for i in range(ThisImageTiles.shape[0]):
                Results_df.at[Im+'_'+str(i),'ClassifierConf']=PredClass[i,1]
            CC+=1
        ModelCount=ModelCount+1
    Results_df.to_csv(ResultsPath+'TileClassifier_LORO_final_repeat'+str(Repeat)+'.csv')
   